In [ ]:
from pymongo import MongoClient
import json
import pandas as pd
import numpy as np

In [ ]:
#Establish connection with MongoDB
try :
    client = MongoClient('localhost', 27017)
    print("Connection successful!")
except :
    print("Connection unsuccessful!")
#Establish connection
# client = MongoClient('localhost', 27017)
# db = client['mentalStateDatabase']
# collection = db['mental+StateCollection']

In [ ]:
#Read Json data
filepath = r'C:\Users\Pratiksha Chate\Desktop\Books\DAP\Project\mental_state.json'     #read JSON data
try :
    with open(filepath) as file:
        file_data = json.load(file)    #load json dada
    print("File loaded!")
except :
    print("Unable to open the file!")
finally :
    file.close()    #file closed
    print("File closed!")

In [ ]:
try :
    db = client['mentalStateDatabase']    # Create DB with DB name as mentalStateDatabase
    collection = db['mentalStateCollection']    #Create collection : mentalStateCollection to store JSON data
    collection.insert_one(file_data)          #insert the loaded file into collection
except :
    print("Cannot access DB or Collection, unable to insert file!") #Error
finally :
    client.close()    #Close connection

In [ ]:
getData = pd.DataFrame(list(collection.find()))    #storing the data into the dataframe

columns = [] #Creating an empty list to store column names
#Fetching Column names details from data stored in MongoDB using for loop 
for data in getData["meta"]:
    for column in data['view']['columns']:
        columnName = column['name']
        if columnName not in columns:
            columns.append(columnName)
        
print(columns) 

In [ ]:
mentalStateData = pd.DataFrame(getData["data"][0], columns = columns)    #Creating data frame to store data in the respective columns
mentalStateData.head()      # Viewing the data once all the columns are stored

In [ ]:
mentalStateData

In [ ]:
#Dropping the first eight columns as they are relevant only when dealing with json files.
mentalStateData.drop(mentalStateData.columns[0:8], axis = 1, inplace = True)
pd.DataFrame().from_dict(mentalStateData)
mentalStateData

In [ ]:
#Counting the missing values in each column
missing = pd.DataFrame(mentalStateData.isnull().sum())
print(missing)

In [ ]:
#Replacing the missing values of columns Value, Low CI, High CI with median value
mentalStateData['Value'] = mentalStateData['Value'].fillna(mentalStateData['Value'].median(axis=0))
mentalStateData['Low CI'] = mentalStateData['Low CI'].fillna(mentalStateData['Low CI'].median(axis=0))
mentalStateData['High CI'] = mentalStateData['High CI'].fillna(mentalStateData['High CI'].median(axis=0))

In [ ]:
missing = pd.DataFrame(mentalStateData.isnull().sum())
print(missing)
mentalStateData

In [ ]:
# Filling in the missing value with '0.0-0.0'
mentalStateData.loc[mentalStateData['Confidence Interval'].isnull(), 'Confidence Interval'] = "0.0-0.0"
myList = [i.split('-') for i in  mentalStateData['Confidence Interval'][mentalStateData['Confidence Interval'].notnull()]]
#print(myList)

mean1 =[]    #creating an empty list for mean
for i in myList: # For loop to iterate through mylist which holds upper and lower range value of confidence interval.
    mean = (float(i[0]) + float(i[1]))/2     #Calculating the mean of LHS and RHS of the range
    mean1.append(mean)                       #Append the newly calculated mean in the empty list
mentalStateData['CIRange'] = pd.DataFrame(mean1)    #Assign the mean1 list to CIRange

print(mentalStateData['CIRange'])

In [ ]:
# Updating the '0.00' value with median value
mentalStateData.loc[ mentalStateData['CIRange'] == 0 | np.isnan(mentalStateData['CIRange']), 'CIRange' ] = mentalStateData['CIRange'].median()

In [ ]:
#Ensure that there are no missing value in the Confidence Interval column
missing = pd.DataFrame(mentalStateData.isnull().sum())
print(missing)

In [ ]:
# Quartile Range column values will also be filled my the mean of the range value

mentalStateData.loc[mentalStateData['Quartile Range'].isnull(), 'Quartile Range'] = "0.0-0.0" # Filling missing record with 0 value
myList1 = [i.split('-') for i in  mentalStateData['Quartile Range'][mentalStateData['Quartile Range'].notnull()]]

mean2 =[] # Creating an empty list
for i in myList1:
    mean = (float(i[0]) + float(i[1]))/2
    #print(mean)
    mean2.append(mean) # adding the mean of the Quartile range values to the empty mean2 list.
    
   
 #Creating a new  column name QR and adding the mean value of Quartile Range
mentalStateData['QR'] = pd.DataFrame(mean2)

#print(mentalStateData['QR'])

In [ ]:
#Replacing 0 values with median value
mentalStateData.loc[ mentalStateData['QR'] == 0 | np.isnan(mentalStateData['QR']), 'QR' ] = mentalStateData['QR'].median()
#mentalStateData['QR']

In [ ]:
missing = pd.DataFrame(mentalStateData.isnull().sum())
print(missing)

In [ ]:
#Dropping original Confidence Interval and Quartile Range columns as the range values have been handled and stored under new columns CIRange and QR respectively
mentalStateData.drop(['Confidence Interval','Quartile Range'], axis = 1, inplace = True)
pd.DataFrame().from_dict(mentalStateData)

In [ ]:
mentalStateData.rename(columns={'Indicator':'Indicator','Group':'Groups','State':'State','Subgroup':'Subgroup','Phase':'Phase','Time Period':'TimePeriod','Time Period Label':'TimePeriodLabel','Time Period Start Date':'TimePeriodStartDate','Time Period End Date':'TimePeriodEndDate','Value':'Value','Low CI':'LowCI','High CI':'HighCI','CIRange':'ConfidenceInterval','QR':'QuartileRange'
}, inplace=True)

In [ ]:
mentalStateData.to_csv('mental_state.csv',index=False)